# Modelos de Pronóstico para Negocios

En este notebook aprenderás a aplicar modelos de pronóstico (forecasting) utilizando bases de datos educativas. Exploraremos desde los conceptos básicos hasta la implementación de algoritmos clásicos de series de tiempo, con ejemplos prácticos y visualizaciones.

---

## 1. Importar librerías y clonar repositorio de bases de datos

Comenzaremos importando las librerías necesarias y clonando el repositorio educativo que contiene bases de datos en formato SQLite.

In [1]:
# Importar librerías principales
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, adfuller
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

# Clonar el repositorio de bases de datos educativas (solo la primera vez)
!git clone https://github.com/davidjamesknight/SQLite_databases_for_learning_data_science.git

# Cambiar al directorio del repositorio
%cd SQLite_databases_for_learning_data_science

Cloning into 'SQLite_databases_for_learning_data_science'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 39 (delta 21), reused 34 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 2.10 MiB | 6.53 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/workspaces/forecasting-641600/SQLite_databases_for_learning_data_science


## 2. Explorar tablas y buscar series de tiempo

Nos conectaremos a la base de datos y listaremos las tablas disponibles. Buscaremos alguna tabla que contenga datos de series de tiempo (por ejemplo, columnas de fecha y valores numéricos).

In [2]:
# Conectarse a la base de datos flights.db
db = sqlite3.connect('flights.db')
cursor = db.cursor()

# Listar todas las tablas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()
print("Tablas encontradas:")
for tabla in tablas:
    print("-", tabla[0])

# Revisar las primeras filas de cada tabla para buscar series de tiempo
for tabla in tablas:
    nombre_tabla = tabla[0]
    print(f"\nPrimeras filas de la tabla '{nombre_tabla}':")
    df_temp = pd.read_sql_query(f"SELECT * FROM {nombre_tabla} LIMIT 5", db)
    display(df_temp)

Tablas encontradas:
- Observation
- Month

Primeras filas de la tabla 'Observation':


,year,passengers,month_id
0,1949,112,0
1,1949,118,1
2,1949,132,2
3,1949,129,3
4,1949,121,4



Primeras filas de la tabla 'Month':


,month_id,month
0,0,January
1,1,February
2,2,March
3,3,April
4,4,May


## 3. Antecedentes y generalidades de los algoritmos para forecasting

El **pronóstico** (forecasting) es el proceso de estimar valores futuros basados en datos históricos. Es fundamental en negocios para anticipar ventas, demanda, inventarios, precios, entre otros.

**Aplicaciones típicas:**
- Predicción de ventas mensuales o anuales.
- Estimación de demanda de productos.
- Planeación de recursos y presupuestos.
- Proyecciones financieras.

Los algoritmos de forecasting pueden ser simples (promedios móviles) o avanzados (modelos estadísticos y de machine learning).

## 4. Objetivos del desarrollo de pronósticos en la toma de decisiones

Los pronósticos permiten a las empresas:
- Optimizar inventarios y reducir costos de almacenamiento.
- Mejorar la planeación financiera y presupuestal.
- Anticipar cambios en la demanda y ajustar estrategias de marketing.
- Tomar decisiones informadas sobre producción, compras y recursos humanos.

Un buen pronóstico ayuda a minimizar riesgos y aprovechar oportunidades en mercados dinámicos.

## 5. Identificar y preparar una serie de tiempo

Seleccionaremos una tabla adecuada, extraeremos una columna de fecha y una de valores, y prepararemos los datos para análisis de series de tiempo.

In [3]:
# Query explícito: unir Observation, Month y Airport para obtener una serie de tiempo de pasajeros mensuales

sql = """
SELECT
    O.year,
    M.month,
    O.passengers
FROM
    Observation AS O
JOIN
    Month AS M ON O.month_id = M.month_id
ORDER BY
    O.year, M.month_id
"""

df = pd.read_sql_query(sql, db)

# Crear columna de fecha
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'], format='%Y-%B')
df = df.sort_values('date')
df.set_index(
    'date',
    inplace=True
)
df_ts = df['passengers']
df.head()

,year,month,passengers
date,,,
1949-01-01,1949,January,112
1949-02-01,1949,February,118
1949-03-01,1949,March,132
1949-04-01,1949,April,129
1949-05-01,1949,May,121


## 6. Visualización de la serie de tiempo (Diagnóstico)

Graficaremos la serie de tiempo para observar tendencias, estacionalidad y posibles anomalías.

In [4]:
# TODO: Código para hacer gráfico de línea
fig=px.line(
    df_ts,
    title='pasajeros mensuales de vuelos'
)

fig.update_layout(
    xaxis_title='fecha',
    yaxis_title='pasajeros',
)

### 6.1. Descomposición de series de tiempo

La descomposición permite separar la serie en componentes de **tendencia**, **estacionalidad** y **residuales**. Esto ayuda a entender mejor el comportamiento de la serie.

---

#### 📊 Modo Aditivo
El modelo se expresa como: 
$$Y_t = T_t + S_t + R_t$$

1. **Tendencia ($T_t$)**  
   Se obtiene aplicando una media móvil centrada con ventana igual al período estacional (ej. 12 meses).  
   Esto suaviza la serie y muestra la dirección a largo plazo.

2. **Estacionalidad ($S_t$)**  
   - Se calcula primero la serie detrendida:  
     $$Y_t - T_t$$
   - Luego se agrupan los valores por posición dentro del ciclo (ej. todos los eneros, febreros, etc.) y se promedian.  
   - El resultado es un patrón estacional promedio que se replica en toda la serie.

3. **Residuo ($R_t$)**  
   Es lo que queda después de quitar tendencia y estacionalidad:  
   
   $$R_t = Y_t - T_t - S_t$$

---

#### 📈 Modo Multiplicativo
El modelo se expresa como:
$$Y_t = T_t \cdot S_t \cdot R_t$$

1. **Tendencia ($T_t$)**  
   Igual que en el aditivo: media móvil centrada.

2. **Estacionalidad ($S_t$)**  
   - Se calcula primero la serie detrendida relativa:  
     $$\frac{Y_t}{T_t}$$
   - Luego se agrupan los valores por posición dentro del ciclo y se promedian.  
   - El resultado es un patrón estacional relativo (factores multiplicativos) que se replica en toda la serie.

3. **Residuo ($R_t$)**  
   Es el cociente entre la serie y los componentes calculados:  
   $$R_t = \frac{Y_t}{T_t \cdot S_t}$$

---

#### 🔑 Diferencia clave
- En el **modo aditivo**, la estacionalidad es una magnitud constante que se suma o resta.  
- En el **modo multiplicativo**, la estacionalidad es un factor proporcional que escala con el nivel de la serie.  


In [5]:
# TODO: Código para hacer gráfico de descomposición
result = seasonal_decompose(
    df_ts,
    model='multiplicative',
    period=12
)

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, subplot_titles=['Serie original', 'Tendencia', 'Estacionalidad', 'Residuo'])

fig.add_trace(go.Scatter(x=result.observed.index, y=result.observed, name='Original'), row=1, col=1)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, name='Tendencia'), row=2, col=1)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, name='Estacionalidad'), row=3, col=1)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, name='Residuo'), row=4, col=1)

fig.update_layout(height=900, title_text='Descomposición de la serie de tiempo')
fig.show()

### 6.2. Análisis de Autocorrelación

La descomposición permite separar la serie en componentes de tendencia, estacionalidad y residuales. Esto ayuda a entender mejor el comportamiento de la serie.

**Fórmula de la Autocorrelación**

**📌 Versión expandida (forma desarrollada)**
    - Equivalente a:

$$
\rho_k = \frac{\sum_{t=k+1}^{n} (Y_t - \bar{Y})(Y_{t-k} - \bar{Y})}{\sum_{t=1}^{n} (Y_t - \bar{Y})^2}
$$


---

**📌 Versión compacta (con covarianza)**
    - La autocorrelación en el rezago \(k\) se define como:

$$
\rho_k = \frac{\text{Cov}(Y_t, Y_{t-k})}{\sigma_{Y_t} \cdot \sigma_{Y_{t-k}}}
$$

In [6]:
# TODO: Crear gráfico de barras para los valores de autocorrelación
acf_vals=acf(df_ts, nlags=48)
lags = list(range(len(acf_vals)))

fig=go.Figure()
fig.add_trace(
    go.Bar(
        x=lags,
        y=acf_vals,
        name='Autocorrelacion'
    )
)

## 7. Algoritmos de forecasting para series de tiempo

Los principales algoritmos de forecasting se clasifican en:

- **Modelos clásicos:** Promedios móviles, suavización exponencial, ARIMA, SARIMA, Holt-Winters.
- **Modelos de machine learning:** Regresión, Random Forest, XGBoost, redes neuronales.
- **Modelos avanzados:** Prophet (Facebook), modelos de descomposición, modelos híbridos.

En este notebook nos enfocaremos en los modelos clásicos, ideales para datos de series de tiempo univariadas.

## 8. Métodos de promedios móviles

Los promedios móviles son una técnica simple pero efectiva para suavizar las fluctuaciones de corto plazo en una serie de tiempo y resaltar tendencias a largo plazo. Se basan en calcular el promedio de un número fijo de puntos de datos recientes.

- **Promedio móvil simple (SMA):** Calcula el promedio aritmético de los últimos *n* valores de la serie de tiempo. Cada punto de datos en la ventana de tiempo tiene el mismo peso.

  **Fundamento matemático:**
  El SMA para un período *n* en el tiempo *t* se calcula como:
  $$SMA_t = \frac{P_t + P_{t-1} + ... + P_{t-n+1}}{n}$$
  Donde:
  - $P_t$ es el valor de la serie de tiempo en el tiempo *t*.
  - *n* es el número de períodos en la ventana del promedio móvil.

  **Fundamento estadístico:**
  El SMA es una estimación de la media local de la serie de tiempo. Reduce el ruido aleatorio y suaviza la serie, haciendo que las tendencias sean más visibles. Sin embargo, puede rezagarse detrás de los cambios bruscos en la tendencia.

- **Promedio móvil ponderado (WMA):** Asigna diferentes pesos a los valores dentro de la ventana de tiempo, generalmente dando más peso a los datos más recientes. Esto permite que el WMA reaccione más rápidamente a los cambios en la serie que el SMA.

  **Fundamento matemático:**
  El WMA para un período *n* en el tiempo *t* se calcula como:
  $$WMA_t = \frac{w_n P_t + w_{n-1} P_{t-1} + ... + w_1 P_{t-n+1}}{w_n + w_{n-1} + ... + w_1}$$
  Donde:
  - $P_t$ es el valor de la serie de tiempo en el tiempo *t*.
  - $w_i$ es el peso asignado al valor en el tiempo $t-i+1$. Los pesos suelen disminuir linealmente o exponencialmente a medida que los datos se vuelven más antiguos.

  **Fundamento estadístico:**
  El WMA es también una estimación de la media local, pero con un énfasis en los datos recientes. La elección de los pesos es crucial y depende de la naturaleza de la serie y del objetivo del análisis. Un WMA con pesos decrecientes linealmente es un caso común.

A continuación, implementamos el promedio móvil simple.

In [15]:
# TODO: Ajusta media movil y crear gráfico de línea doble
# Aquí podemos hacer una función y reutilizarlo.

df["SMA_12"] = df_ts.rolling(window=12).mean()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_ts.index,
        y=df_ts,
        mode="lines",
        name="pasajeros reales",
        opacity=0.5
    )
)

fig.add_trace(
    go.Scatter(
        x=df_ts.index,
        y=df["SMA_12"],
        mode="lines",
        name="Promedio movil de 12 meses"
    )
)

fig.update_layout(
    title="promedio movil de 12 meses",
    xaxis_title="Fecha",
    yaxis_title="numero de pasajeros"
)
fig.show()

## 9. Métodos de suavización exponencial

La **suavización exponencial** es una técnica de pronóstico que asigna pesos que disminuyen exponencialmente a medida que los datos se vuelven más antiguos. Esto permite que los modelos se adapten mejor a cambios recientes en la serie de tiempo.


Estos métodos son ampliamente usados por su simplicidad y eficacia.


Existen varios métodos de suavización exponencial, adecuados para diferentes tipos de series:

### **9.1. Suavización exponencial simple (SES):** Para series sin tendencia ni estacionalidad.
  
  **Fundamento matemático:**
  La fórmula de recurrencia para SES es:
  $$F_{t+1} = \alpha Y_t + (1 - \alpha) F_t$$
  Donde:
  - $F_{t+1}$ es el pronóstico para el período $t+1$.
  - $Y_t$ es el valor real de la serie en el período $t$.
  - $F_t$ es el pronóstico para el período $t$.
  - $\alpha$ es el parámetro de suavización (0 < $\alpha$ < 1).
  
  **Fundamento estadístico:**
  SES es equivalente a un modelo ARIMA(0,1,1) sin constante. Asume que el nivel de la serie cambia aleatoriamente con el tiempo y que los errores de pronóstico son ruido blanco. Un valor de $\alpha$ cercano a 1 da más peso a las observaciones recientes, mientras que un valor cercano a 0 da más peso al historial pasado.

Aplicamos el método de suavización exponencial simple usando `statsmodels`. Es ideal para series sin tendencia ni estacionalidad marcada.

In [18]:
# TODO: Suavización Exponencial Simple
%cd /workspaces/forecasting-641600/
import calculations as calcs


/workspaces/forecasting-641600


In [21]:
ses_model = SimpleExpSmoothing(df_ts).fit(
    smoothing_level=0.2,
    optimized=False
)
df['SES'] = ses_model.fittedvalues

calcs.graficar_serie(
    original=df_ts,
    forecast=df['SES'],
    title='suavización exponencial simple'
)

### **9.2. Doble suavización exponencial (Método de Brown):** Para series con tendencia lineal pero sin estacionalidad. Utiliza dos ecuaciones de suavización para estimar el nivel y la tendencia.

  **Fundamento matemático:**
  Las fórmulas de recurrencia para el método de Brown son:
  Nivel suavizado simple: $$S_t = \alpha Y_t + (1 - \alpha) S_{t-1}$$
  Nivel doblemente suavizado: $$S'_t = \alpha S_t + (1 - \alpha) S'_{t-1}$$
  La tendencia se estima como: $$b_t = \frac{\alpha}{1-\alpha} (S_t - S'_t)$$
  El pronóstico para *h* períodos futuros es: $$F_{t+h} = a_t + h b_t$$
  Donde:
  - $S_t$ es el nivel suavizado simple en el tiempo *t*.
  - $S'_t$ es el nivel doblemente suavizado en el tiempo *t*.
  - $a_t = S_t + b_t$ es el nivel estimado en el tiempo *t*.
  - $b_t$ es la tendencia estimada en el tiempo *t*.
  - $\alpha$ es el parámetro de suavización (0 < $\alpha$ < 1).

  **Fundamento estadístico:**
  El método de Brown es un caso especial del método de Holt. Asume que la serie tiene una tendencia lineal aditiva y que el nivel y la pendiente cambian aleatoriamente. Es equivalente a un modelo ARIMA(0,2,2) sin constante.

In [22]:
# TODO: Brown Model
brown_model = ExponentialSmoothing(
    df_ts,
    trend='add',
    seasonal=None
).fit()

df['Brown'] = brown_model.fittedvalues

calcs.graficar_serie(
    df_ts,
    df.Brown,
    title='Método de Brown'
)

### **9.3. Método de Holt:** Extiende la suavización exponencial simple para manejar series con tendencia lineal (aditiva o multiplicativa), pero sin estacionalidad. Utiliza dos parámetros de suavización: uno para el nivel ($\alpha$) y otro para la tendencia ($\beta$).

  **Fundamento matemático:**
  Las fórmulas de recurrencia para el método de Holt (con tendencia aditiva) son:
  Nivel: $$L_t = \alpha Y_t + (1 - \alpha) (L_{t-1} + b_{t-1})$$
  Tendencia: $$b_t = \beta (L_t - L_{t-1}) + (1 - \beta) b_{t-1}$$
  El pronóstico para *h* períodos futuros es: $$F_{t+h} = L_t + h b_t$$
  Donde:
  - $L_t$ es el nivel estimado en el tiempo *t*.
  - $b_t$ es la tendencia estimada en el tiempo *t*.
  - $\alpha$ es el parámetro de suavización del nivel (0 < $\alpha$ < 1).
  - $\beta$ es el parámetro de suavización de la tendencia (0 < $\beta$ < 1).

  **Fundamento estadístico:**
  El método de Holt es equivalente a un modelo ARIMA(0,2,2) cuando se utiliza tendencia aditiva. Permite que el nivel y la tendencia se ajusten a los cambios en la serie.

In [23]:
# TODO: Holt Model
holt_model=Holt(df_ts).fit()
df['Holt']=holt_model.fittedvalues

calcs.graficar_serie(
        df_ts,
        df.Holt,
        title='Método de Holt'
    )

### **9.4. Método de Holt-Winters:** Permite modelar series con tendencia y estacionalidad (aditiva o multiplicativa). Agrega un tercer parámetro de suavización ($\gamma$) para el componente estacional.

  **Fundamento matemático:**
  Las fórmulas de recurrencia para el método de Holt-Winters (con tendencia y estacionalidad aditivas) son:
  Nivel: $$L_t = \alpha (Y_t - S_{t-p}) + (1 - \alpha) (L_{t-1} + b_{t-1})$$
  Tendencia: $$b_t = \beta (L_t - L_{t-1}) + (1 - \beta) b_{t-1}$$
  Estacionalidad: $$S_t = \gamma (Y_t - L_t) + (1 - \gamma) S_{t-p}$$
  El pronóstico para *h* períodos futuros es: $$F_{t+h} = L_t + h b_t + S_{t+h-p}$$
  Donde:
  - $L_t$ es el nivel estimado en el tiempo *t*.
  - $b_t$ es la tendencia estimada en el tiempo *t*.
  - $S_t$ es el componente estacional estimado en el tiempo *t*.
  - $\alpha$, $\beta$, $\gamma$ son los parámetros de suavización (0 < $\alpha$, $\beta$, $\gamma$ < 1).
  - *p* es el período estacional (por ejemplo, 12 para datos mensuales con estacionalidad anual).

  **Fundamento estadístico:**
  El método de Holt-Winters (aditivo) es equivalente a un modelo SARIMA(0,1,p)(0,1,0)p. Modela la serie como la suma de un nivel, una tendencia y un componente estacional periódico, más un error aleatorio. Permite que el nivel, la tendencia y la estacionalidad se adapten a los cambios en la serie. La elección entre modelos aditivos y multiplicativos depende de si la magnitud de la estacionalidad es independiente o proporcional al nivel de la serie.

In [24]:
# TODO: Holt-Winters Model
hw_model = ExponentialSmoothing(
    df_ts,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit()

df['HW']=hw_model.fittedvalues

calcs.graficar_serie(
    df_ts,
    df['HW'],
    title='Método de Holt Winters'
)

## 11. Aplicación del algoritmo

A continuación, un ejemplo de ajuste y predicción usando statsmodels:

In [25]:
from sklearn.metrics import mean_squared_error

# TODO: Calcular el RMSE de cada método
# function - Para cada método, alinear los índices antes de calcular el error
mape_sma = calcs.mape(df_ts,df['SMA_12'])
print(mape_sma)

10.7785191375832


In [ ]:
modelos = df.drop(
    columns=['year','month','passengers']
).columns.tolist()

In [28]:
for mdl in modelos:
    print(mdl, calcs.mape(df_ts, df[mdl]))
    print()

SMA_12 10.7785191375832

SES 11.490871041192847

Brown 8.938428622430914

Holt 9.075544737547498

HW 3.534355746091125



In [29]:
# TODO: Pronóstico futuro (14 meses)
n_forecast=14

forecast = hw_model.forecast(n_forecast)

forecast

1961-01-01    451.576310
1961-02-01    427.256384
1961-03-01    464.696765
1961-04-01    500.101255
1961-05-01    508.658613
1961-06-01    567.711517
1961-07-01    647.894966
1961-08-01    632.458259
1961-09-01    534.371342
1961-10-01    484.927927
1961-11-01    418.455746
1961-12-01    471.054591
1962-01-01    482.708586
1962-02-01    458.388660
Freq: MS, dtype: float64

In [30]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['passengers'], mode='lines', name='Pasajeros reales'))
fig.add_trace(go.Scatter(x=df.index, y=df['HW'], mode='lines', name='Holt-Winters (ajuste)'))
future_dates = pd.date_range(df.index[-1]+pd.DateOffset(months=1), periods=n_forecast, freq='MS')
fig.add_trace(go.Scatter(x=future_dates, y=forecast, mode='lines+markers', name='Pronóstico futuro', line=dict(dash='dash')))
fig.update_layout(title='Pronóstico de pasajeros con Holt-Winters', xaxis_title='Fecha', yaxis_title='Pasajeros')
fig.show()

In [31]:
# TODO: Agregar intervalos de confianza.
residuals = df['passengers'] - df['HW']
sigma = np.std(residuals)

# Intervalos de confianza (ejemplo 95%)
z = 1.96
forecast_upper = forecast + z * sigma
forecast_lower = forecast - z * sigma

# Banda superior e inferior
fig.add_trace(
go.Scatter(
x=future_dates,
y=forecast_upper,
mode='lines',
name='IC 95% superior',
line=dict(color='gray', dash='dot')
)
)

fig.add_trace(
go.Scatter(
x=future_dates,
y=forecast_lower,
mode='lines',
name='IC 95% inferior',
line=dict(color='gray', dash='dot')
)
)

fig.update_layout(
title='Pronóstico de pasajeros con Holt-Winters + Intervalos de confianza',
xaxis_title='Fecha',
yaxis_title='Pasajeros'
)

fig.show()

## Conclusión

A continuación, se presenta un resumen de cuándo utilizar cada modelo de pronóstico según las características de la serie de tiempo:

| Modelo                     | Características de la serie                       | Uso recomendado                          |
|----------------------------|--------------------------------------------------|-----------------------------------------|
| Suavización Exponencial Simple (SES) | Sin tendencia ni estacionalidad                  | Series estables sin patrones claros    |
| Método de Brown/Holt      | Con tendencia lineal, sin estacionalidad         | Series con crecimiento o decrecimiento  |
| Holt-Winters              | Con tendencia y estacionalidad                   | Series con patrones estacionales claros |

Este resumen proporciona una guía rápida para seleccionar el modelo de pronóstico más adecuado según las características de los datos.